# Search for AGN in the Extended Chandra Deep Field South and show their Light Curves

- Confluence page : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/372867091/System-level+Science+Performance+Verification+Sprint
- slides : https://docs.google.com/presentation/d/1bPjS5NGtiEL2lfjmvP1UYdG_tMEDfZxX26ayhf7MhtY/edit#slide=id.ga2f7fb1a1f_0_70

- where to find the campains
- https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

- plot Navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator/plot/%2Frepo%2Fmain/LSSTComCam%2Fruns%2FDRP%2FDP1%2Fw_2025_05%2FDM-48666/objectTableCore_coaddInputCount_SkyPlot

- Notebooks examples
- https://github.com/lsst-dm/DMTR-401/blob/main/notebooks/test_LVV-T40_T1240.ipynb
- https://github.com/lsst-dm/DMTR-412/blob/tickets/DM-38728/notebooks/test_LVV-T1751_AM1_AM2.ipynb

- author : Sylvie Dagoret-Campagne
- creation date : 2025-05-11
- last update : 2025-05-11
- Redo Visits like here : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/Visits/stat_on_visits_LSSTComCamSim.ipynb
- To find what I did on LSSTComCamSim : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/LightCurves/MultiColor_lightCurves-DMRehearsal2024_01-AuxTel-DZPOnCCD.ipynb
- **Confluence page** : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/443613290/Science+Verification+Sprint+Feb+2025


In [ ]:
# Confirm that the version of the Science Pipelines is recent:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom
from lsst.geom import SpherePoint, degrees
import lsst.afw.display as afwDisplay

In [ ]:
afwDisplay.setDefaultBackend("firefly")

In [ ]:
import os
import gc
import glob
import numpy as np
import pandas as pd
import random

import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Reselect Data Release 3, default
Gaia.ROW_LIMIT = 100000
from astropy.visualization import (MinMaxInterval, AsinhStretch, ZScaleInterval, LogStretch, LinearStretch,
                                   ImageNormalize)

In [ ]:
import seaborn as sns
from itertools import cycle, islice

In [ ]:
from astropy.time import Time
from datetime import datetime, timedelta

In [ ]:
import lsst
import lsst.geom as geom
from lsst.geom import Angle
from lsst.geom import SpherePoint
from lsst.geom import AngleUnit

In [ ]:
def find_tract_patch_and_display(butler, skymap,ra_deg, dec_deg, band, framecount):
    """
    - butler
    - skymap
    - ra_deg, dec_deg : source
    - band
    - framecount
    """
    # Ouvrir le dépôt
    #butler = Butler(repo)
    #skymap = butler.get("skyMap")
    band_to_color = {"u":"blue","g":"green","r":"red","i":"orange","z":"yellow","y":"purple"}

    datasettype = "objectTable_tract"
    therefs = butler.registry.queryDatasets(datasettype,  collections=collection)

    # Créer le point céleste
    coord = SpherePoint(ra_deg * degrees, dec_deg * degrees)


    found = False
    # loop only on existing tracts
    
    for ref in therefs:
        tract_id = ref.dataId["tract"]
        tractInfo = skymap[tract_id]
        if tractInfo.contains(coord):
            patchInfo = tractInfo.findPatch(coord)
            patch_id = patchInfo.getIndex()  # Tuple (x, y)
            patch_indexseq = patchInfo.getSequentialIndex()
            print(f"Tract: {tract_id}, Patch: {patch_indexseq}")
            found = True
            break
    # Charger l’image coadd correspondante
    dataId = {"tract": tract_id, "patch": patch_indexseq, "band": band,"skymap": skymapName}
    #deepCoadd_calexp
    coadd = butler.get("deepCoadd_calexp", dataId)
    # Récupération du WCS
    wcs = coadd.getWcs()

    datatitle = f"deepcoadd-LSSTComCam : {tract_id} , {patch_indexseq} , {band}"
    print(datatitle)

    # Affichage avec marqueur
    
    display = afwDisplay.Display(frame = framecount)
    display.scale('asinh', 'zscale')
    #afw_display.scale('linear', min=zmin, max=zmax)
    #display.setImageColormap(cmap='grey')
    display.setImageColormap(cmap='Grey_r')
    display.mtv(coadd.getImage(),title=datatitle)

    # Création du point céleste
    pix_point = wcs.skyToPixel(coord)
    x, y = pix_point.x,pix_point.y
    
    col = band_to_color[band] 
    display.dot("+", x, y, size=50, ctype=col)
    

    
    return tract_id, patch_indexseq, band

In [ ]:
# https://pipelines.lsst.io/modules/lsst.geom/getting-started.html
func_degToRad = lambda x : Angle(x,lsst.geom.degrees).asRadians()

In [ ]:
# Set plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
from matplotlib import colors
zscale = ZScaleInterval()

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 24})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 5})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})


In [ ]:
def nJy_to_ab_mag(f_njy):
    """Convert scalar or array flux in nJy to AB magnitude."""
    f_njy = np.asarray(f_njy)
    mag = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = f_njy > 0
    mag[mask] = -2.5 * np.log10(f_njy[mask]) + 31.4
    return mag


def nJy_err_to_ab_err(f_njy, f_err):
    """Propagate flux error to magnitude error."""
    f_njy = np.asarray(f_njy)
    f_err = np.asarray(f_err)
    mag_err = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = (f_njy > 0) & (f_err > 0)
    mag_err[mask] = (2.5 / np.log(10)) * (f_err[mask] / f_njy[mask])
    return mag_err


In [ ]:
def ab_mag_to_nJy(mag_ab):
    """Convert AB magnitude to flux in nanojanskys."""
    return 10 ** ((31.4 - mag_ab) / 2.5)


## 1) Build the table of AGN from the Vizier Catalog

In [ ]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
ECDFS_RA = 53.1
ECDFS_DEC = -27.8
ECDFS_COORD = SpherePoint(ECDFS_RA, ECDFS_DEC , degrees)

In [ ]:
# Coordonnées approximatives du centre du E-CDFS
coord = SkyCoord(ra=ECDFS_RA, dec = ECDFS_DEC, unit=(u.deg, u.deg), frame='icrs')
radius = 15 * u.arcmin

# Exemple avec le catalogue Luo+2017 (7Ms CDF-S)
catalog_id = "J/ApJS/228/2"
result = Vizier(columns=["*"]).query_region(coord, radius=radius, catalog=catalog_id)

In [ ]:
sources_table = result['J/ApJS/228/2/sources']
print(sources_table.colnames)

In [ ]:
# On calcule le log10 de LX
logLX = np.log10(sources_table['LX'])

# Et on l'ajoute comme nouvelle colonne à la table
sources_table['logLX'] = logLX

# Attention au nom exact de la colonne
logLX = sources_table['logLX']
is_agn = logLX > 42

is_agn = sources_table['logLX'] > 42  # AGN si LX > 10^42 erg/s
agn_table = sources_table[is_agn]

In [ ]:
agn_table[['RAJ2000', 'DEJ2000', 'logLX', 'LX', 'zspec']].pprint(max_lines=10)

In [ ]:
galaxy_table = sources_table[sources_table['OType'] == 'Galaxy']

In [ ]:
galaxy_table[['RAJ2000', 'DEJ2000', 'logLX', 'LX', 'zspec']].pprint(max_lines=10)

## Plot info from AGN Vizier catalog

In [ ]:
df_galaxy = galaxy_table.to_pandas()
df_agn = agn_table.to_pandas()

In [ ]:
plt.figure(figsize=(14,5))

plt.subplot(1, 2, 1)
plt.hist(df_agn['zspec'].values, bins=20, color='green', alpha=0.7)
plt.xlabel("Redshift zspec")
plt.ylabel("Nombre de sources")
plt.title("Distribution en redshift")

plt.subplot(1, 2, 2)
plt.hist(df_agn['logLX'].values, bins=20, color='orange', alpha=0.7)
plt.xlabel("log(Lx) [erg/s]")
plt.ylabel("Nombre de sources")
plt.title("Distribution de la luminosité X")

plt.tight_layout()
plt.show()


## Initialisation of rubin science pipeline
- Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

### Configuration

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = '/repo/main'
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666' # work
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810' # work
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359' # work


# bad : crash collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'

# bad : collection = "LSSTComCam/runs/DRP/20241101_20241211/w_2024_51/DM-48233"

# not working perhaps because I am using w_2025_10 version
# bad : no ccd visit collection = "LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864"
# bad : no ccd visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_15/DM-50050'
# bad : no cce visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864'
# bad : no cce visit collection collection = 'LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751'


instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = \'" + instrument+ "\'"
collectionStr = collection.replace("/", "_")

In [ ]:
FLAG_DUMP_COLLECTIONS = False
FLAG_DUMP_DATASETS = False
FLAG_DUMP_OBJECTSTABLECOLUMNS = False
FLAG_CUT_OBJECTSMAG = True
FLAG_CUT_OBJECTSSNR = True

In [ ]:
MAGCUT = 24.0
SNRCUT = 5.0

In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
all_bands_colors = ["blue", "green", "red", "orange", "yellow", "purple"]

### Initialisation of Butler

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
skymap = butler.get('skyMap', skymap=skymapName, collections=collection)

In [ ]:
camera = butler.get("camera", collections=collection, instrument=instrument)

In [ ]:
# 4. Trouver dans quel tract/patch se trouve la coordonnée
ECDFS_tract_info = skymap.findTract(ECDFS_COORD)

In [ ]:
print('Tract number for ECDFS ::',ECDFS_tract_info.getId())

In [ ]:
TRACTSEL = ECDFS_tract_info.getId()

In [ ]:
print(f"TRACT : {TRACTSEL}")

In [ ]:
# Check here the collections available
if FLAG_DUMP_COLLECTIONS:
    for _ in sorted(registry.queryCollections(expression=instrument + "/*")):
        if "/calib/" not in _ and "u/" not in _:
            print(_)

In [ ]:
if FLAG_DUMP_DATASETS:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection_validation).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and ("Plot" not in datasetType.name)
                and ("Metric" not in datasetType.name)
                and ("metric" not in datasetType.name)
                and (("Table" in datasetType.name) or ("Zeropointp" in datasetType.name) or ("fgcm" in datasetType.name) or ("transm" in datasetType.name) or ("Transm" in datasetType.name)
                or ("source" in datasetType.name) or ("Source" in datasetType.name) or ("object" in datasetType.name) or ("Object" in datasetType.name))
            ):
                
                print(datasetType)

### Get list of Tracts and Patches

- Just to know which tracts are involved in the observations

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype,  collections=collection)

In [ ]:
tractsId_list = np.unique([ref.dataId['tract'] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

## Select AGN which are in the selected Patch

In [ ]:
df_agn[df_agn["zspec"]<1.5]

In [ ]:
galaxyindex_selected = []
N =len(df_galaxy)
for idx in range(N):
    ra_target,dec_target = df_galaxy.iloc[idx][['RAJ2000','DEJ2000']]
    target_point = SpherePoint(ra_target, dec_target, degrees)
    if ECDFS_tract_info.contains(target_point):
        galaxyindex_selected.append(idx)         

In [ ]:
print(galaxyindex_selected)

In [ ]:
agnindex_selected = []
N =len(df_agn)
for idx in range(N):
    ra_target,dec_target = df_agn.iloc[idx][['RAJ2000','DEJ2000']]
    target_point = SpherePoint(ra_target, dec_target, degrees)
    if ECDFS_tract_info.contains(target_point):
        agnindex_selected.append(idx)         

In [ ]:
print(agnindex_selected)

In [ ]:
index_agn_selected = 0
index_galaxy_selected = 0
FLAG_AGN_SELECTED  = False

In [ ]:
if FLAG_AGN_SELECTED: 
    ra_target,dec_target = df_agn.iloc[index_agn_selected ][['RAJ2000','DEJ2000']]
    target_point = SpherePoint(ra_target, dec_target, degrees)
    target_coord = SkyCoord(ra=ra_target*u.deg, dec=dec_target*u.deg)
    logLx = df_agn.iloc[index_agn_selected ]['logLX']
    zspec = df_agn.iloc[index_agn_selected ]['zspec']
    target_title = f"agn :: {index_agn_selected}, log L(erg/s) = {logLx:.2f} , zspec = {zspec:.2f}, (ra,dec) = {ra_target:.5f},{dec_target:.5f}"
else:
    ra_target,dec_target = df_galaxy.iloc[index_galaxy_selected ][['RAJ2000','DEJ2000']]
    target_point = SpherePoint(ra_target, dec_target, degrees)
    target_coord = SkyCoord(ra=ra_target*u.deg, dec=dec_target*u.deg)
    logLx = df_galaxy.iloc[index_galaxy_selected ]['logLX']
    zspec = df_galaxy.iloc[index_galaxy_selected ]['zspec']
    target_title = f"galaxy :: {index_galaxy_selected}, log L(erg/s) = {logLx:.2f} , zspec = {zspec:.2f}, (ra,dec) = {ra_target:.5f},{dec_target:.5f}"
    

## Find in which patch is the target

In [ ]:
patch_info = ECDFS_tract_info.findPatch(target_point)
tractNbSel = ECDFS_tract_info.getId()
patchNbSel =  patch_info.getSequentialIndex()
where_clause = f"skymap = '{skymapName}' AND tract = {tractNbSel} AND patch = {patchNbSel}"
print(where_clause)

## Define the Stack DM product used for the DIA analysis

In [ ]:
datasettype =  'forcedSourceOnDiaObjectTable'
#datasettype =  'diaSourceTable_tract'
#datasettype = 'goodSeeingDiff_assocSsSrcTable' # bad table
therefs = butler.registry.queryDatasets(datasettype,  collections=collection,where=where_clause)

In [ ]:
for count,ref in enumerate(therefs):
    the_id = ref.dataId
    print(the_id)
    break

In [ ]:
if FLAG_AGN_SELECTED: 
    figname1 = f"agn_{index_agn_selected}_{datasettype}_v1.png"
    figname2 = f"agn_{index_agn_selected}_{datasettype}_v2.png"
else:
    figname1 = f"galaxy_{index_agn_selected}_{datasettype}_v1.png"
    figname2 = f"galaxy_{index_agn_selected}_{datasettype}_v2.png"

In [ ]:
target_coord

In [ ]:
TRACTSEL

In [ ]:
all_tables = []

for count,ref in enumerate(therefs):
    the_id = ref.dataId
    the_tract_id = the_id["tract"] 
    # select the visits in that Tract (ECDFS)
    if the_tract_id == TRACTSEL: 
        catalog = butler.get(ref)
        #catalog["dateobs"] = catalog.apply(lambda x: x['visit']//100_000, axis=1)
        #catalog["seq"] = catalog["visit"] - catalog["dateobs"]*100_000
        #t["ra_rad"] = t["coord_ra"].apply(func_degToRad)  
        #t["ra_dec"] = t["coord_dec"].apply(func_degToRad)  
        ra = catalog["coord_ra"].values
        dec = catalog["coord_dec"].values
        # Créer tous les points du catalogue
        #points = [SpherePoint(r, d, degrees) for r, d in zip(ra, dec)]
        catalog_coords = SkyCoord(ra=ra*u.deg, dec=dec*u.deg)

        # Calculer la distance angulaire à la cible (en arcsec)
        #distances_arcsec = np.array([pt.separation(target_point).asArcseconds() for pt in points])
        # 5. Calcul vectorisé des distances (résultat en arcsec)
        distances_arcsec = target_coord.separation(catalog_coords).arcsecond
        catalog["sep"] = distances_arcsec

        # 5. Sélectionner les objets proches (ex: < 1.5 arcsec)
        if datasettype ==  'forcedSourceOnDiaObjectTable':
            radius_arcsec = 3
        elif datasettype ==  'diaSourceTable_tract':
            radius_arcsec = 5
        elif datasettype == 'goodSeeingDiff_assocSsSrcTable':
            radius_arcsec = 5
        else:
            radius_arcsec = 3

        # select
        nearby = catalog[distances_arcsec < radius_arcsec]
        
        if len(nearby) > 0 :
            #print(nearby)
            all_tables.append(nearby)  
            #break

        #if(len(all_tables)>10):
        #    break
   

In [ ]:
len(all_tables)

In [ ]:
t = all_tables[0]

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,4))
t["sep"].hist(bins=50,ax=ax)
plt.show()

In [ ]:
t = t[t["sep"]<3.5]

In [ ]:
t

### Split visit into dayobs and seq

In [ ]:
t["dateobs"] = t.apply(lambda x: x['visit']//100_000, axis=1)
t["seq"] = t["visit"] - t["dateobs"]*100_000

In [ ]:
t["seq"]

In [ ]:
t["mags"] = t["psfFlux"].apply(lambda flux : -2.5 * np.log10(flux) + 31.4)
t["valid"] = (t["psfFlux"] >0) & (~t["psfFlux_flag"])
visit_list = t["visit"].astype(int)

#### Find the time associated to a visit

In [ ]:
# On interroge la table visitDefinition
rows = registry.queryDimensionRecords("visit", where=f"visit in {tuple(visit_list)}")

# 4. Construire un tableau des résultats
results = []
for row in rows:
    visit_id = row.id

    # Extraire l'instant de début de l'observation (Time astropy)
    start_time = row.timespan.begin

    # Convertir en MJD et ISO
    mjd = start_time.to_value("mjd")  # Ex: 60384.28718
    isot = start_time.to_value("isot")  # Ex: '2024-04-19 06:53:32.000'
    
    #mjd = row.startDate.toMjd()
    #utc = Time(mjd, format='mjd', scale='utc').to_value('iso')
    results.append({"visit": visit_id, "mjd": mjd, "isot": isot})

df_times = pd.DataFrame(results).sort_values("visit")
df_times.set_index("visit",inplace=True)

In [ ]:
t["mjd"] = t["visit"].apply(lambda x: df_times.loc[x]["mjd"])
t["isot"] = t["visit"].apply(lambda x: df_times.loc[x]["isot"])

In [ ]:
tb = {}
for band in ["u", "g", "r", "i", "z", "y"]:
    tb[band] = t[t["band"] == band]

## Plot light curves

In [ ]:
band_to_col = {"u":"b", "g":"g", "r":"r", "i":"orange", "z":"grey", "y":"k"}

In [ ]:
fig,axs = plt.subplots(6,1,figsize=(18,16),sharex=True,layout="constrained")

for idx,band in enumerate(["u", "g", "r", "i", "z", "y"]):
    ax =axs[idx]
    ax.scatter(tb[band].mjd, tb[band].mags,color=band_to_col[band])
    ax.set_ylabel("mag_psfFlux")
    if idx==0:
        ax.set_title(target_title,fontweight = "bold")
ax.set_xlabel("mjd")

subtitle = collectionStr + "_" + datasettype
plt.suptitle(subtitle)
#plt.tight_layout()
plt.savefig(figname1)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,8),sharex=True,layout="constrained")

for idx,band in enumerate(["u", "g", "r", "i", "z", "y"]):
    ax.scatter(tb[band].mjd, tb[band].mags,color=band_to_col[band])

ax.set_ylabel("mag_psfFlux")
ax.set_title(target_title,fontweight = "bold")
ax.set_xlabel("mjd")

subtitle = collectionStr + "_" + datasettype
plt.suptitle(subtitle)
#plt.tight_layout()
plt.savefig(figname2)
plt.show()

## Plot the deepCoadds

In [ ]:

framecount = 0
for iband,band in enumerate(all_bands):
    framecount +=1
    find_tract_patch_and_display(butler, skymap,ra_target, dec_target, band,framecount=framecount)